In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import os
import pyspark

scala_version = '2.12'  # TODO: Ensure this is correct
spark_version = pyspark.__version__

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.5.1'
]

args = os.environ.get('PYSPARK_SUBMIT_ARGS', '')
if not args:
    args = f'--packages {",".join(packages)}'
    print('Using packages', packages)
    os.environ['PYSPARK_SUBMIT_ARGS'] = f'{args} pyspark-shell'
else:
    print(f'Found existing args: {args}') 

spark = SparkSession.builder\
  .master("spark://spark-master:7077") \
  .appName("kafka-example")\
  .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
  .config("spark.eventLog.enabled", "true") \
  .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
  .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
  .getOrCreate()

sc =spark.sparkContext



Using packages ['org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5', 'org.apache.kafka:kafka-clients:3.5.1']
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-946ee9d4-a966-459d-b438-b026d9810122;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.5 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.5 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.1 in central
	found com.github.luben#zstd-jni;1.5.5-1 in centra

In [7]:
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka-1:9092") \
  .option("subscribe", "entrada") \
  .load()
df_str = df.selectExpr("CAST(value AS STRING) as value")
df_str

DataFrame[value: string]

[Stage 3:===============>                                        (55 + 2) / 200]

In [9]:
from pyspark.sql.functions import col
words = df_str.select(explode(split(col("value"), " ")).alias("word"))
word_counts = words.groupBy("word").count()
output = word_counts.selectExpr(
    "CAST(word AS STRING) AS key",
    "CAST(count AS STRING) AS value"
)
output

DataFrame[key: string, value: string]

[Stage 3:==========================>                             (95 + 2) / 200]

In [4]:
 # Start running the query that prints the running counts to the console
query = word_counts \
          .writeStream \
          .outputMode("complete") \
          .format("memory") \
          .queryName("consulta2") \
          .start()


[Stage 1:>                                                        (0 + 2) / 200]

In [10]:
query = output.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-1:9092") \
    .option("topic", "salida2") \
    .option("checkpointLocation", "/user/jovyan/kafka-wordcount-checkpoint5") \
    .outputMode("update") \
    .start()


[Stage 3:=========>     (126 + 2) / 200][Stage 5:>                (0 + 0) / 200]

In [5]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM consulta2').show())
    sleep(5)

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+--------+-----+
|    word|count|
+--------+-----+
|   guapa|    1|
|caracola|    2|
|  bonita|    1|
|     que|    1|
|    hola|    6|
|     tal|    1|
+--------+-----+



None

KeyboardInterrupt: 

In [ ]:
from IPython.display import display, clear_output
from time import sleep
clear_output(wait=True)
display(query.status)
display(spark.sql('SELECT * FROM consulta1').show())



In [ ]:
from pyspark.sql.functions import to_json
kafka_output_topic = "salida"

kafka_output_config = {
    "kafka.bootstrap.servers": "kafka-1:9092",  # Coloca aquí los servidores de arranque de Kafka
    "topic": kafka_output_topic
}

query2 = wordCounts \
    .selectExpr("CAST(word AS STRING) AS key", "to_json(struct(*)) AS value") \
    .writeStream \
    .format("kafka") \
    .outputMode("complete") \
    .options(**kafka_output_config) \
    .option("checkpointLocation", "/home/jovyan/checkpoints") \
    .start()


In [ ]:
query.awaitTermination()